# Query Embeddings

This notebook demonstrates how embeddings can be used to query the survey
dataset. The raw data is read as a pandas data frame; the `search_display()`
function prints text responses in the data frame that are most similar to a given
prompt. This assumes embeddings have already been generated using
`embeddings.py`.

In [1]:
import pandas as pd
from lib import search_display

In [2]:
# import raw survey data as a dataframe
df = pd.read_csv("data/reflections-public.csv")

In [3]:
# search using mxbai and display 10 results
search_display("I am confident about my writing", df, limit=10, model="mxbai")

(distance: 8.50, perm: Student121) I feel very confident about my writing skills.

(distance: 10.4, perm: Student1084) I personally feel very confident in my writing as a whole. In the first student writing by Desert Ahana it seems as if the writer was told to look a t a certain scenario and provide evidence of both sides of the situation and hare their opinion on the topic. I believe that in my past classes I have had the ability to do writings similar to this one such as one  specific writing assignment where we took legal cases and paired them with poems in order to determine if the rulings in court reflected the natural feelings often held within human capacity, This first writing seemed much more relaxed, and as if the individual was in front of a group of peers talking about a subject, in which I feel confident in writing similarly. Although in my English classes I have not done a ton of similar writings similar to Mao's I believe other classes have prepared me. In my IB Psychology class we were to do an internal assessment where we were to look at a certain phenomenon and describe it with our own research and those of

(distance: 10.7, perm: Student3236) As a writer, I believe that I already have my own voice in my own style. Also, the process I've used for writing has been quite effective for me, and I feel very confident in it. Combining this with my comfortability with the Writing 2 writing samples and academic essays on this writing program test, I believe that I will be a better fit in Writing 2

(distance: 10.8, perm: Student1181) I feel extremely confident in my writing and reading abilities. I came from a very large and academically rigorous school, Canyon Crest Academy, which regularly ranks as the top public high school in California. Still, within that school I was one of the strongest writers. I scored highest on essays in most of my english classes and my essays were commonly used as examples. In addition, I was admitted to an after school humanities based program in which I strengthened my reading and writing skills even further. I feel that I would do well in writing 2 where I would work on my own writing and research. I also feel confident that I would be able to handle the scholarly writing that is analyzed in that course. I feel excited not intimidated by studying high-level works and retargeting the content to an intended audience. I think going directly into Writing 2 is the best fit for me.

(distance: 10.9, perm: Student742) I am very comfortable and confident with myself as a writer and I think that I will be able to excel and learn in Writing 2. With that said, I still have a lot to learn about university-level reading and writing but with the skills I have developed in high school, I think I am adequately prepared to tackle what Writing 2 has to offer. With my six years of reading and writing experience at Marlborough School as well as being an AP Lit student, I have ample experience responding to a text-based question, crafting my own thesis statement/claim, and defending that claim with evidence. 

(distance: 11.0, perm: Student1217) I am confident in my current writing ability, as well as my ability to learn new writing concepts and ideas. Most of the things in this survey that I am less familiar with such as argumentative essays,  rhetorical analysis, and analyzing sources are covered in writing 2. Whereas things in writing 1 such as structure, tone, and diction I am already confident in. 

(distance: 11.0, perm: Student812) As I recently stated in one of my responses, I still have room for improvement when it comes to writing like a reader/reading like a writer. Although I am proud of my improvements and took this assessment convinced I was ready for Writing 2, a little doubt arose. Of course, I know I am capable of anything and will handle Writing 2, I want to be 100% confident and honestly feel like I need a little refresher since I missed out on lots of learning this past year (zoom classes were not helpful).

(distance: 11.1, perm: Student857) In the past I've done similar writings however its been awhile yet I'm still confident in my ability to write writings such as these. Past writing assignments and readings have increased my knowledge and ability to produce such writings that I find my self confident in doing so. I think majority of my own writings have been similar to these pieces of writing. From my past writing assignments we've went over similar formats of writing and using various techniques and skills picked up throughout years of writing to complete assignments such as these.

(distance: 11.1, perm: Student1842) While I may seem overly confident, I believe I am ready to begin a more competitive class like Writing 2. After hearing the testimony of the two girls in Writing 2, I believe that it is a more high level class that will teach me things that I haven’t learned in my high school such as interpreting high level papers. I’ve had a lot of practice writing with rhetoric and interpreting speeches and articles of various lengths and genres and believe I am ready to learn and practice what I’ve learned to apply it to more competitive lessons.

(distance: 11.1, perm: Student333) I have confidence in my level of english writing and comprehension. I do not feel the need to start with an entry level writing.

In [4]:
# search using openai embeddings. The OpenAI API is used to get the prompt embedding.
search_display("I am confident about my writing", df, limit=10, model="openai")

(distance: 0.50, perm: Student121) I feel very confident about my writing skills.

(distance: 0.84, perm: Student3236) As a writer, I believe that I already have my own voice in my own style. Also, the process I've used for writing has been quite effective for me, and I feel very confident in it. Combining this with my comfortability with the Writing 2 writing samples and academic essays on this writing program test, I believe that I will be a better fit in Writing 2

(distance: 0.85, perm: Student1217) I am confident in my current writing ability, as well as my ability to learn new writing concepts and ideas. Most of the things in this survey that I am less familiar with such as argumentative essays,  rhetorical analysis, and analyzing sources are covered in writing 2. Whereas things in writing 1 such as structure, tone, and diction I am already confident in. 

(distance: 0.87, perm: Student169) I am quite confident in my writing ability, as I went to a large high school and took all honors level English classes except for my senior year where that was not an option. My senior teacher did tell me I should have been in AP Literature. I have an extensive reading and writing background, and feel comfortable in my writing style and ability. I have extensive experience writing in various genres and formats, varying from articles to multiple page analytical essays. I feel that Writing 2 is the best fit for me. 

(distance: 0.87, perm: Student857) My confidence in my writing experience is high as well as my knowledge regarding essays , essay formats nd kills and writing techniques. However I believe that there is still room to learn and grow to become a better writer and have a better understanding of several more topics addressed in the survey questions. Also I think my comfort level would be higher knowing I can regain and build off the knowledge taught to me in Writing 1 as well as it can help he succeed and thrive in Writing 2.

(distance: 0.88, perm: Student1084) I personally feel very confident in my writing as a whole. In the first student writing by Desert Ahana it seems as if the writer was told to look a t a certain scenario and provide evidence of both sides of the situation and hare their opinion on the topic. I believe that in my past classes I have had the ability to do writings similar to this one such as one  specific writing assignment where we took legal cases and paired them with poems in order to determine if the rulings in court reflected the natural feelings often held within human capacity, This first writing seemed much more relaxed, and as if the individual was in front of a group of peers talking about a subject, in which I feel confident in writing similarly. Although in my English classes I have not done a ton of similar writings similar to Mao's I believe other classes have prepared me. In my IB Psychology class we were to do an internal assessment where we were to look at a certain phenomenon and describe it with our own research and those of

(distance: 0.88, perm: Student2338) My own writing compared to these students writing is different and similar. I am very opinionated and use my personal experience and morals when writing which I noticed most of these writers had. My previous experiences, I would say, have prepared me a lot for this specific type of writing. This writing is one of the areas I am more confident when it comes to my writing. I have had similar pieces like these in the past that I have either read or written.

(distance: 0.88, perm: Student1105) Although I have not produced many pieces of writing similar to the examples, I feel quite confident in my ability to write them. I mostly experienced this style of writing when I began writing my college application essays. This helped me become more comfortable with writing about my own experiences in connection to the topic I needed to write about. 

(distance: 0.89, perm: Student1181) I feel extremely confident in my writing and reading abilities. I came from a very large and academically rigorous school, Canyon Crest Academy, which regularly ranks as the top public high school in California. Still, within that school I was one of the strongest writers. I scored highest on essays in most of my english classes and my essays were commonly used as examples. In addition, I was admitted to an after school humanities based program in which I strengthened my reading and writing skills even further. I feel that I would do well in writing 2 where I would work on my own writing and research. I also feel confident that I would be able to handle the scholarly writing that is analyzed in that course. I feel excited not intimidated by studying high-level works and retargeting the content to an intended audience. I think going directly into Writing 2 is the best fit for me.

(distance: 0.89, perm: Student333) I have confidence in my level of english writing and comprehension. I do not feel the need to start with an entry level writing.